In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("")))))

In [2]:
BASE_PATH = r'C:\Users\com\Documents\GitHub\level2-competitiveds-recsys-06\data\etc'
data = pd.read_csv('add_grid_id.csv')
park = pd.read_csv(os.path.join(BASE_PATH,'parkinfo.csv'))
subway = pd.read_csv(os.path.join(BASE_PATH,'subwayinfo.csv'))
school = pd.read_csv(os.path.join(BASE_PATH,'schoolinfo.csv'))

---

In [3]:
park = park.groupby(['latitude','longitude'])['area'].mean().reset_index()
school = school.groupby(['latitude','longitude']).max().reset_index()
subway = subway.groupby(['latitude','longitude']).max().reset_index()

In [4]:
park["park_idx"] = park.index
school['school_idx'] = school.index
subway['subway_idx'] = subway.index

In [5]:
print(park.head())
print(subway.head())
print(school.head())

    latitude   longitude      area  park_idx
0  31.373335  126.954253    2159.0         0
1  33.218571  126.252728    1733.0         1
2  33.229983  126.498677    7461.0         2
3  33.236820  126.562841  104137.0         3
4  33.243482  126.569065   48279.0         4
    latitude   longitude  subway_idx
0  36.769502  126.951108           0
1  36.777629  127.052991           1
2  36.780483  127.003249           2
3  36.792053  127.104361           3
4  36.793759  127.121400           4
    latitude   longitude schoolLevel  school_idx
0  33.169187  126.269883  elementary           0
1  33.219962  126.253233      middle           1
2  33.225592  126.255688  elementary           2
3  33.229189  126.253314        high           3
4  33.231926  126.259500        high           4


In [6]:
# 그리드 크기 설정 (3km = 약 0.027도)
grid_size = 0.027  # 3km를 위도/경도 단위로 변환한 크기

# data에서 위도와 경도의 최소값 및 최대값을 계산
lat_min = data['latitude'].min()
lat_max = data['latitude'].max()
lon_min = data['longitude'].min()
lon_max = data['longitude'].max()

# 그리드 경계 설정 (data에서 추출한 최소, 최대 위도/경도 값을 사용)
grid_lat_min = lat_min
grid_lat_max = lat_max
grid_lon_min = lon_min
grid_lon_max = lon_max

In [7]:
from tqdm import tqdm
import pandas as pd

# 그리드 생성
grid = []

# 위도, 경도 그리드 범위를 계산
lat_steps = int((grid_lat_max - grid_lat_min) / grid_size) + 1
lon_steps = int((grid_lon_max - grid_lon_min) / grid_size) + 1

# tqdm 적용하여 진행 상황 표시
for i in tqdm(range(lat_steps), desc="Latitude steps"):
    for j in tqdm(range(lon_steps), desc="Longitude steps", leave=False):
        grid.append({
            'grid_id': f'{i}-{j}',
            'min_lat': grid_lat_min + i * grid_size,
            'max_lat': grid_lat_min + (i + 1) * grid_size,
            'min_lon': grid_lon_min + j * grid_size,
            'max_lon': grid_lon_min + (j + 1) * grid_size
        })

grid_df = pd.DataFrame(grid)


Longitude steps:   0%|          | 0/44 [00:00<?, ?it/s]

Latitude steps: 100%|██████████| 47/47 [00:00<00:00, 120.84it/s]


In [8]:
# 각 시설의 위치를 그리드에 매칭하는 함수
def find_grid(lat, lon, grid_df):
    grid_row = grid_df[
        (grid_df['min_lat'] <= lat) & (grid_df['max_lat'] > lat) &
        (grid_df['min_lon'] <= lon) & (grid_df['max_lon'] > lon)
    ]
    if not grid_row.empty:
        return grid_row.iloc[0]['grid_id']
    else:
        return None

In [9]:
# 공원 데이터에 그리드 매칭
park['grid_id'] = None  # 먼저 빈 grid_id 열을 생성
for index, row in tqdm(park.iterrows(), total=len(park), desc="Matching park grids"):
    park.at[index, 'grid_id'] = find_grid(row['latitude'], row['longitude'], grid_df)

# 지하철 데이터에 그리드 매칭
subway['grid_id'] = None  # 먼저 빈 grid_id 열을 생성
for index, row in tqdm(subway.iterrows(), total=len(subway), desc="Matching subway grids"):
    subway.at[index, 'grid_id'] = find_grid(row['latitude'], row['longitude'], grid_df)

# 학교 데이터에 그리드 매칭
school['grid_id'] = None  # 먼저 빈 grid_id 열을 생성
for index, row in tqdm(school.iterrows(), total=len(school), desc="Matching school grids"):
    school.at[index, 'grid_id'] = find_grid(row['latitude'], row['longitude'], grid_df)

# 결과 확인
print(park.head())
print(subway.head())
print(school.head())

Matching school grids: 100%|██████████| 11425/11425 [00:07<00:00, 1431.67it/s]

    latitude   longitude      area  park_idx grid_id
0  31.373335  126.954253    2159.0         0    None
1  33.218571  126.252728    1733.0         1    None
2  33.229983  126.498677    7461.0         2    None
3  33.236820  126.562841  104137.0         3    None
4  33.243482  126.569065   48279.0         4    None
    latitude   longitude  subway_idx grid_id
0  36.769502  126.951108           0    None
1  36.777629  127.052991           1    None
2  36.780483  127.003249           2    None
3  36.792053  127.104361           3    None
4  36.793759  127.121400           4    None
    latitude   longitude schoolLevel  school_idx grid_id
0  33.169187  126.269883  elementary           0    None
1  33.219962  126.253233      middle           1    None
2  33.225592  126.255688  elementary           2    None
3  33.229189  126.253314        high           3    None
4  33.231926  126.259500        high           4    None


In [10]:
print(park.isnull().sum())
print(subway.isnull().sum())
print(school.isnull().sum())

latitude        0
longitude       0
area            0
park_idx        0
grid_id      9887
dtype: int64
latitude       0
longitude      0
subway_idx     0
grid_id       14
dtype: int64
latitude          0
longitude         0
schoolLevel       0
school_idx        0
grid_id        7244
dtype: int64


In [11]:
# None이 아닌 값에 대해서만 '-'를 제거하고 숫자로 변환하는 함수
def convert_grid_id(grid_id):
    if pd.isna(grid_id):  # grid_id가 None이면 None을 반환
        return None
    else:
        return int(grid_id.replace('-', ''))

# 공원 데이터에 numeric_grid_id 추가
park['numeric_grid_id'] = park['grid_id'].apply(convert_grid_id)

# 지하철 데이터에 numeric_grid_id 추가
subway['numeric_grid_id'] = subway['grid_id'].apply(convert_grid_id)

# 학교 데이터에 numeric_grid_id 추가
school['numeric_grid_id'] = school['grid_id'].apply(convert_grid_id)

# 결과 확인
print(park[['grid_id', 'numeric_grid_id']].head())
print(subway[['grid_id', 'numeric_grid_id']].head())
print(school[['grid_id', 'numeric_grid_id']].head())



  grid_id  numeric_grid_id
0    None              NaN
1    None              NaN
2    None              NaN
3    None              NaN
4    None              NaN
  grid_id  numeric_grid_id
0    None              NaN
1    None              NaN
2    None              NaN
3    None              NaN
4    None              NaN
  grid_id  numeric_grid_id
0    None              NaN
1    None              NaN
2    None              NaN
3    None              NaN
4    None              NaN


In [20]:
# data에 grid_id 추가
data['grid_id'] = None  # 빈 grid_id 열 생성

# tqdm을 사용하여 각 행에 대해 grid_id 매칭
for index, row in tqdm(data.iterrows(), total=len(data), desc="Matching data grids"):
    data.at[index, 'grid_id'] = find_grid(row['latitude'], row['longitude'], grid_df)

Matching data grids: 100%|██████████| 1951400/1951400 [23:37<00:00, 1376.86it/s]


In [12]:
data.columns

Index(['index', '_type', 'contract_year_month', 'contract_day', 'area_m2',
       'contract_date_numeric', 'floor', 'built_year', 'latitude', 'longitude',
       'age', 'contract_0', 'contract_1', 'deposit', 'apt_idx', 'area',
       'grid_deposit', 'nearest_park_distance', 'apt_deposit_rank',
       'apt_area_deposit_rank', 'recent_deposit', 'interest_rate', 'grid_id'],
      dtype='object')

In [13]:
data['grid_id'] = data['grid_id'].apply(convert_grid_id)

---

## merge

In [14]:
from sklearn.neighbors import BallTree

In [15]:
park.rename(columns={'numeric_grid_id': 'park_grid_id'},inplace=True)
subway.rename(columns={'numeric_grid_id': 'subway_grid_id'},inplace=True)
school.rename(columns={'numeric_grid_id': 'school_grid_id'},inplace=True)

In [16]:
print(park.head())
print(subway.head())
print(school.head())

    latitude   longitude      area  park_idx grid_id  park_grid_id
0  31.373335  126.954253    2159.0         0    None           NaN
1  33.218571  126.252728    1733.0         1    None           NaN
2  33.229983  126.498677    7461.0         2    None           NaN
3  33.236820  126.562841  104137.0         3    None           NaN
4  33.243482  126.569065   48279.0         4    None           NaN
    latitude   longitude  subway_idx grid_id  subway_grid_id
0  36.769502  126.951108           0    None             NaN
1  36.777629  127.052991           1    None             NaN
2  36.780483  127.003249           2    None             NaN
3  36.792053  127.104361           3    None             NaN
4  36.793759  127.121400           4    None             NaN
    latitude   longitude schoolLevel  school_idx grid_id  school_grid_id
0  33.169187  126.269883  elementary           0    None             NaN
1  33.219962  126.253233      middle           1    None             NaN
2  33.225592 

In [17]:
temp_df = data.drop_duplicates(subset="apt_idx", keep="first")

EARTH_RADIUS_KM = 6371.0

temp_df_rad = np.radians(temp_df[["latitude", "longitude"]].values)

In [18]:
park_info_rad = np.radians(park[["latitude", "longitude"]].values)
tree = BallTree(park_info_rad, metric="haversine")
distances, indices = tree.query(temp_df_rad, k=1)
temp_df["nearest_park_distance"] = (distances.flatten() * EARTH_RADIUS_KM * 1000)
temp_df["nearest_park_idx"] = (park["park_idx"].iloc[indices.flatten()].values)
temp_df['park_grid_id'] = temp_df['nearest_park_idx'].map(park.set_index('park_idx')['park_grid_id'])
temp_df['park_area'] = temp_df['nearest_park_idx'].map(park.set_index('park_idx')['area'])

C:\Users\com\AppData\Local\Temp\ipykernel_9236\1779847317.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["nearest_park_distance"] = (distances.flatten() * EARTH_RADIUS_KM * 1000)
C:\Users\com\AppData\Local\Temp\ipykernel_9236\1779847317.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["nearest_park_idx"] = (park["park_idx"].iloc[indices.flatten()].values)
C:\Users\com\AppData\Local\Temp\ipykernel_9236\1779847317.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [19]:
temp_df

,index,_type,contract_year_month,contract_day,area_m2,contract_date_numeric,floor,built_year,latitude,longitude,...,grid_deposit,nearest_park_distance,apt_deposit_rank,apt_area_deposit_rank,recent_deposit,interest_rate,grid_id,nearest_park_idx,park_grid_id,park_area
0,0,train,201906,25,84.9981,20190625,9,2019,37.054314,127.045216,...,25581.836691,498.618918,7683,29300,NaN,1.78,520,9759,420.0,3898.0
3,3,train,201907,15,59.3400,20190715,1,1986,36.964647,127.055847,...,18375.213675,169.839678,18366,58901,NaN,1.68,121,9139,121.0,1616.0
4,4,train,201904,12,59.8100,20190412,6,1995,36.972390,127.084514,...,12627.879781,382.401815,18369,58863,NaN,1.85,222,9201,122.0,3986.0
5,5,train,201904,26,84.9342,20190426,13,2016,36.965423,127.048779,...,18375.213675,288.442840,11986,43564,NaN,1.85,121,9150,120.0,4858.3
6,6,train,201905,7,39.2700,20190507,3,1990,36.957089,127.047449,...,4635.616438,272.286038,18382,59031,NaN,1.85,121,9117,121.0,6173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940450,139222,test,202401,13,79.4000,20240113,3,2003,37.540437,127.145738,...,45508.725993,213.004451,5738,22416,NaN,3.66,2324,14384,2324.0,996.3
1940955,139727,test,202401,23,64.1000,20240123,10,2015,37.499548,127.144838,...,53208.361204,205.998864,5774,22578,NaN,3.66,2124,13716,2124.0,10962.5
1944142,142914,test,202401,16,75.6200,20240116,3,2004,37.463234,126.642899,...,22537.737348,600.149610,5759,22449,NaN,3.66,206,12943,205.0,494.0
1944417,143189,test,202403,25,49.9000,20240325,13,2015,37.459061,126.674991,...,18786.523185,450.729668,5724,23181,NaN,3.59,207,12879,207.0,7974.2


In [20]:
data = data.merge(temp_df[[
    'apt_idx','nearest_park_distance','nearest_park_idx','park_grid_id','park_area']]
    ,on=['apt_idx'],how='left')

In [21]:
school_info_rad = np.radians(school[["latitude", "longitude"]].values)
tree = BallTree(school_info_rad, metric="haversine")
distances, indices = tree.query(temp_df_rad, k=1)
temp_df["nearest_school_distance"] = (distances.flatten() * EARTH_RADIUS_KM * 1000)
temp_df["nearest_school_idx"] = (school["school_idx"].iloc[indices.flatten()].values)
temp_df['school_grid_id'] = temp_df['nearest_school_idx'].map(school.set_index('school_idx')['school_grid_id'])

In [22]:
data = data.merge(temp_df[[
    'apt_idx','nearest_school_distance','nearest_school_idx','school_grid_id']]
    ,on=['apt_idx'],how='left')

In [24]:
# subway의 위도와 경도 정보를 radians로 변환 (subway 데이터를 제대로 사용해야 함)
subway_info_rad = np.radians(subway[["latitude", "longitude"]].values)

# BallTree 적용
tree = BallTree(subway_info_rad, metric="haversine")
distances, indices = tree.query(temp_df_rad, k=1)

# indices가 temp_df의 개수와 일치하는지 확인
print(f"indices length: {len(indices)}, temp_df length: {len(temp_df)}")

# 지하철 데이터에 대한 가장 가까운 거리 및 인덱스 정보 추가
temp_df["nearest_subway_distance"] = (distances.flatten() * EARTH_RADIUS_KM * 1000)

# 인덱스가 유효한지 확인하는 코드 추가
valid_indices = indices.flatten()

# 유효한 지하철 인덱스에 따라 nearest_subway_idx 추가 (범위를 벗어난 인덱스를 처리)
temp_df["nearest_subway_idx"] = subway["subway_idx"].iloc[valid_indices].values

# subway_grid_id를 subway_idx에 맞춰 매핑
temp_df['subway_grid_id'] = temp_df['nearest_subway_idx'].map(subway.set_index('subway_idx')['subway_grid_id'])

# 결과 확인
print(temp_df[['nearest_subway_idx', 'subway_grid_id']].head())


indices length: 18676, temp_df length: 18676
   nearest_subway_idx  subway_grid_id
0                  12           521.0
3                  10           222.0
4                  10           222.0
5                  10           222.0
6                  10           222.0


In [25]:
data = data.merge(temp_df[[
    'apt_idx','nearest_subway_distance','nearest_subway_idx','subway_grid_id']]
    ,on=['apt_idx'],how='left')

In [30]:
data.columns

Index(['index', '_type', 'contract_year_month', 'contract_day', 'area_m2',
       'contract_date_numeric', 'floor', 'built_year', 'latitude', 'longitude',
       'age', 'contract_0', 'contract_1', 'deposit', 'apt_idx', 'area',
       'grid_deposit', 'apt_deposit_rank', 'apt_area_deposit_rank',
       'recent_deposit', 'interest_rate', 'grid_id', 'nearest_park_distance',
       'nearest_park_idx', 'park_grid_id', 'park_area',
       'nearest_school_distance', 'nearest_school_idx', 'school_grid_id',
       'nearest_subway_distance', 'nearest_subway_idx', 'subway_grid_id'],
      dtype='object')

In [27]:
data.drop(columns=['nearest_park_distance_x'],inplace=True)

In [29]:
data.rename(columns={'nearest_park_distance_y':'nearest_park_distance'},inplace=True)

In [ ]:
data

In [31]:
data.to_csv('temp_real_v8.csv',index=False)

---